# Bagging与随机森林（Random Forest）详解

Bagging（Bootstrap Aggregating）和随机森林都是**集成学习（Ensemble Learning）**的重要方法，通过组合多个模型来提升准确率、降低方差和防止过拟合。

---

## 一、Bagging（Bootstrap Aggregating）

### 1. 核心思想

Bagging主要用于**降低模型方差**。它通过**有放回随机采样（Bootstrap）**生成多个训练集，分别训练多个基模型，然后将它们的预测结果进行平均（回归）或投票（分类）。

### 2. 算法流程

1. 对原始数据集进行多次有放回采样，生成 $B$ 个不同的Bootstrap样本；
2. 在每个Bootstrap样本上训练一个基学习器（如决策树）；
3. 对新样本预测时：
   - 分类：多数投票
   - 回归：取平均值

公式（回归）：
$$
\hat{f}_{bag}(x) = \frac{1}{B}\sum_{b=1}^B \hat{f}^{(b)}(x)
$$

---

### 3. 优缺点

✅ **优点：**
- 有效减少高方差模型（如决策树）的过拟合
- 简单易实现
- 适用于多种基学习器

❌ **缺点：**
- 并不能降低偏差（如果基模型有偏差）
- 需要更多计算资源

---

## 二、随机森林（Random Forest）

随机森林是在Bagging基础上改进的一种方法。它引入了**特征随机性**，进一步增强模型多样性。

### 1. 核心思想

在Bagging的基础上，随机森林在每棵树节点分裂时：
- **随机选择部分特征子集进行划分**

这样每棵树不仅数据不同，特征也不同，进一步减少相关性。

---

### 2. 算法流程

1. Bootstrap采样生成多个数据子集
2. 对每棵树：
   - 生成一棵完全生长（不剪枝）的决策树
   - 每个节点分裂时随机选取 $m$ 个特征（$m < M$），在这 $m$ 个特征中选择最佳分裂

默认：
- 分类：$m = \sqrt{M}$
- 回归：$m = M/3$

3. 对新样本预测：
   - 分类：多数投票
   - 回归：取平均

---

### 3. 随机森林与Bagging的区别

| 特性               | Bagging                  | 随机森林              |
|--------------------|--------------------------|-----------------------|
| 数据采样           | Bootstrap                | Bootstrap             |
| 特征采样           | 无                       | 每个分裂随机选特征   |
| 基模型             | 任意模型（常用树）       | 决策树（不剪枝）     |
| 相关性降低         | 较低                     | 更低                 |

---

### 4. 优缺点

✅ **优点：**
- 更低方差
- 不易过拟合
- 对缺失值鲁棒
- 可用于特征重要性评估

❌ **缺点：**
- 可解释性差
- 大规模数据训练较慢
- 占用更多内存

---

## 三、Python示例（使用 scikit-learn）

```python
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 加载数据
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Bagging
bagging = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)
bagging.fit(X_train, y_train)
y_pred_bag = bagging.predict(X_test)
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bag))

# Random Forest
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
```

---

## 四、总结

- **Bagging**：通过Bootstrap采样训练多个基模型，提升稳定性和准确性。
- **随机森林**：在Bagging基础上引入特征随机性，使模型更具多样性和抗过拟合能力。
- 随机森林是Bagging的扩展，也是目前最受欢迎的集成方法之一。

---

> **Tip:** 在特征维度较高、模型容易过拟合的场景，推荐使用随机森林。

In [10]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [11]:
iris = load_iris()
X = iris.data
y = iris.target

In [12]:
bag_clf = BaggingClassifier(
    SVC(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    oob_score= True#袋外样本
)

In [13]:
bag_clf.fit(X, y)
y_pred = bag_clf.predict(X)
print(accuracy_score(y, y_pred))
print(bag_clf.oob_score_)

0.9666666666666667
0.96


# 基分类器设置为决策树

In [14]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1
)

In [15]:
bag_clf.fit(X, y)
y_pred = bag_clf.predict(X)
print(accuracy_score(y, y_pred))
# print(bag_clf.oob_score_)

1.0


# 随机森林的API

In [16]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X, y)
y_pred_rf = rnd_clf.predict(X)
print(accuracy_score(y, y_pred_rf))

1.0
